# Model Evauation
Classification models need the following metrics

Cross Validation
- Accuracy
- error

ROC Curve
Steps
- load data
- featurize
- choose model
- pass data and HP's
- run CV
- ROC possible?
- tabulate results
- report

In [248]:
#### Standard Libraries ####
import os
from pprint import pprint
import numpy as np
import pandas as pd
import multiprocessing as mp
from functools import partial
from itertools import product
import timeit
import uuid

#### third-party Libraries ####
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier as SRC
from lolopy.learners import RandomForestClassifier as LRC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#### Local Libraries ####
from utils import (Result, run_k_folds, 
                   report_column_labels,
                   compile_data)
from data_manager import DataManager
from featurizer import Featurizer

In [262]:
# configuration
np.random.seed(8)
load_path = os.path.join('data','training_data.csv')
save_path = os.path.join('results','baseline.csv')
mp_api_key = '7n6DwPUQ5cf8ZTWO'
oversample = False
data_ramp = False
feature_set = ['standard']

## Load data
This object should handle data load and sampling.

Should be able to do the following:
1. load and sample 
2. Run a data ramp

In [263]:
# Load Data
dm = DataManager(load_path, save_path)
dm.load()

'Loaded 2572 records.'


In [264]:
# Sample data
dm.sample_data(100)

In [265]:
# Format and careate composition objects
#dm.compute_formula()
dm.to_binary_classes()
dm.get_pymatgen_composition()
dm.remove_noble_gasses()
dm.remove_features()

In [266]:
f = Featurizer(feature_set)

In [267]:
dm.featurized_data = f.featurize(dm.data)

In [268]:
dm.groups = dm.data['group']

In [269]:
# Set training labels
dm.outputs = dm.data['stable']

In [270]:
k_folds = partial(run_k_folds, inputs=dm.featurized_data, outputs=dm.outputs, groups=dm.groups, sampling=oversample, ramp=data_ramp)

In [271]:
# set models
models = [GaussianNB(), SVC(), SRC(), LogisticRegression(), DummyClassifier(strategy="most_frequent")]
#models = [GaussianNB()]

In [272]:
%%capture
pool = mp.Pool(processes=mp.cpu_count())

In [273]:
%%capture
start_time = timeit.default_timer()
results = pool.map(k_folds, models)    
elapsed = timeit.default_timer() - start_time

In [274]:
compiled = []
for result in results:
    compiled.extend(compile_data(result))
res_df = pd.DataFrame(compiled, columns=report_column_labels)
res_df.to_csv(save_path)
res_df

,type,data_size,accuracy,accuracy_std,f1,f1_std,recall,recall_std,precision,precision_std
0,GaussianNB,871,0.776015,0.083716,0.449831,0.070163,0.539860,0.128882,0.411505,0.101495
1,SVC,871,0.833208,0.030400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,RandomForestClassifier,871,0.915012,0.022541,0.673610,0.080569,0.532389,0.088506,0.931623,0.076861
3,LogisticRegression,871,0.874861,0.056559,0.636372,0.113778,0.627497,0.108862,0.672184,0.158903
4,DummyClassifier,871,0.833208,0.030400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Single model test

In [220]:
import numpy
import pandas
from sklearn.utils import resample
from collections import Counter

def local_oversample(train, outputs):
    majority_class, minority_class = Counter(outputs[train]).most_common()
    minority = train[numpy.where(outputs[train] == minority_class[0])[0]]
    majority = train[numpy.where(outputs[train] == majority_class[0])[0]]
    minority_upsampled = resample(minority,
                              replace=True,
                              n_samples=len(majority),
                              random_state=8)
    t = numpy.append(majority, minority_upsampled)
    pprint(len(t))
    return (t, minority, majority, minority_upsampled)

In [236]:
model = SRC()
ovs = False

In [237]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import KFold, GroupKFold, LeavePGroupsOut, LeaveOneGroupOut
inputs = dm.featurized_data
outputs = dm.outputs
groups = dm.groups
gkf = GroupKFold(n_splits=10)
lpgo = LeavePGroupsOut(n_groups=1)
logo = LeaveOneGroupOut()
splits = list(gkf.split(inputs, outputs, groups=groups.values))
#splits = list(logo.split(inputs, outputs, groups))

In [238]:
train, test = splits[3]

In [239]:
%%capture
# Place all data into df's for easy analysis
train_df = pandas.DataFrame(dm.data.ix[train, ['formula','group','stable']])
test_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])

In [240]:
if ovs:
    train, minority, majority, minority_upsampled = local_oversample(train, outputs)

In [242]:
# Run the ML
model.fit(inputs[train], outputs[train])
prediction = model.predict(inputs[test])
acc = accuracy_score(outputs[test], prediction)
f1s = [precision_recall_fscore_support(
    outputs[test], prediction, labels=[0,1], average='binary')]

/home/mdavidson/.conda/envs/tecca/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



In [243]:
%%capture
# Place all data into df's for easy analysis
train_df = pandas.DataFrame(dm.data.ix[train, ['formula','group','stable']])
test_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])
pred_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])
pred_df['prediction'] = prediction
acc_df = pandas.DataFrame([acc], columns=['accuracy'])

f1_df = pandas.DataFrame(
        f1s, columns=['precision', 'recall', 'f1', 'support'])
ov_df = pd.DataFrame([train_df['group'].isin(test_df['group']).any()], columns=['overlap'])

try:
    lab_df = pandas.DataFrame([pred_df['prediction'].value_counts().values], columns=['pred_0','pred_1'])
except:
    lab_df = pandas.DataFrame([pred_df['prediction'].value_counts().values], columns=['pred_0'])
if ovs:
    sample_df = pandas.DataFrame([train_df['stable'].value_counts().values], columns=['smpl_0','smpl_1'])
else:
    sample_df = pandas.DataFrame([], columns=['sample'])
# Get the accuracy for each label
#ac_lab_df = pandas.DataFrame([test_df['stable'].value_counts().values], columns=['acc_0','acc_1'])

In [244]:
#res = f1_df.join(acc_df).join(ov_df).join(lab_df).join(ac_lab_df).join(sample_df)

In [245]:
res = f1_df.join(acc_df).join(ov_df).join(lab_df).join(sample_df)

In [246]:
#res.to_csv('results/final_modle.csv', index=False)

In [247]:
res

,precision,recall,f1,support,accuracy,overlap,pred_0,pred_1,sample
0,0.697531,0.43295,0.534279,None,0.904229,False,1895,162,NaN


In [255]:
import pickle
#pickle.dump(model, open('rfc.sav', 'wb+'))